**Алгоритм подготовки отчета Валовая прибыль в маркетплейсах по месяцу** 

Алгоритм подготавливает файл с итоговой валовой прибылью за месяц в маркетплейсах по брендам.

Все конфиденциальные данные изменены.

Отчет в Гугл таблице: <https://docs.google.com/spreadsheets/d/1YzV63TYdA7qRpUf0xzX-zm7UDS31ejGwB8pVdOv-mtI/edit#gid=415566582>

In [1]:
# Импортируем нужные библиотеки, добавим переменные и прочитаем датафреймы
import pandas as pd
import datetime
pd.set_option('display.float_format', '{:.2f}'.format)
ozon = pd.read_excel('ozon.xlsx')
wb = pd.read_excel('wb.xlsx')
sku = pd.read_excel('sku.xlsx')
cost_price = pd.read_excel('Себестоимости.xlsx')
det_1 = pd.read_excel('детализация_1.xlsx')
det_2 = pd.read_excel('детализация_2.xlsx')
det_3 = pd.read_excel('детализация_3.xlsx')
det_4 = pd.read_excel('детализация_4.xlsx')
det_5 = pd.read_excel('детализация_5.xlsx')
det_6 = pd.read_excel('детализация_6.xlsx')
det_7 = pd.read_excel('детализация_7.xlsx')
det_8 = pd.read_excel('детализация_8.xlsx')
det_9 = pd.read_excel('детализация_9.xlsx')
det_10 = pd.read_excel('детализация_10.xlsx')
wb_commissions = pd.read_excel('комиссии wb.xlsx')

In [2]:
# Посчитаем усредненные суммы доставки и хранения на штуку товара в wb
det = (
    det_1.append(det_2)
    .append(det_3)
    .append(det_4)
    .append(det_5)
    .append(det_6)
    .append(det_7)
    .append(det_8)
    .append(det_9)
    .append(det_10)
)
wb_cost_of_delivery = wb_commissions['Сумма логистики'].sum()
wb_storage_cost = wb_commissions['Сумма хранения'].sum()
wb_cost_of_delivery_mean = wb_cost_of_delivery / det['Кол-во'].sum()
wb_storage_cost_mean = wb_storage_cost / det['Кол-во'].sum()

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_2189/4088586201.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  det_1.append(det_2)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_2189/4088586201.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  det_1.append(det_2)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_2189/4088586201.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  det_1.append(det_2)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_2189/4088586201.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  det_1.append(det_2)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykern

In [3]:
# Обновим себестоимости
sku = pd.merge(sku, cost_price, how='left', on=('Номенклатура'))
sku['Себестоимость новая'] = sku['Себестоимость новая'].fillna(sku['Себестоимость'])
sku = (
    sku.drop(columns = ['Наименование товара', 'Себестоимость'],axis = 1)
    .rename(columns={"Себестоимость новая": "Себестоимость"})
    .drop_duplicates()
)

In [4]:
# Обработаем датафрейм ozon
# Взаимоудалим возвраты
ss = (
    ozon.query('`За продажу или возврат до вычета комиссий и услуг` < 0')['Номер отправления или идентификатор услуги']
    .unique().tolist()
)
ozon = ozon.query('`Номер отправления или идентификатор услуги` not in @ss')

# Удалим нули в столбце "За продажу или возврат до вычета комиссий и услуг"
ozon = ozon.query('`За продажу или возврат до вычета комиссий и услуг` != 0')

# Оставим нужные столбцы
ozon_use_columns = [
    'Дата начисления', 
    'Артикул', 
    'Название товара или услуги', 
    'Количество', 
    'Итого']
ozon = ozon[ozon_use_columns]

# Приведем столбец "Артикул" к числовому формату
ozon['Артикул'] = ozon['Артикул'].astype(int)

# Добавим по артикулу дополнительные поля
ozon = pd.merge(ozon, sku, how='left', on=('Артикул'))

# Добавим столбец с валовой прибылью
ozon['Валовая прибыль'] = ozon['Итого'] - (ozon['Себестоимость'] * ozon['Количество'])

In [5]:
# Обработаем датафрейм wb
# Оставим нужные столбцы
wb_use_columns = [
    'Дата', 
    'Артикул WB', 
    'Название', 
    'Продажи (WB)', 
    'Суммарная выручка (WB)']
wb = wb[wb_use_columns]

# Удалим нули и отрицательные значения в столбце "Суммарная выручка (WB)"
wb = wb.query('`Суммарная выручка (WB)` > 0').rename(columns={"Артикул WB": "Артикул"})

# Приведем столбец "Дата" к временному формату
wb['Дата'] = wb['Дата'].astype('datetime64')

# Добавим по артикулу дополнительные поля
wb = pd.merge(wb, sku, how='left', on=('Артикул'))

# Добавим столбец "Усредненная сумма доставки на штуку товара"
wb['Усредненная сумма доставки на штуку товара'] = wb_cost_of_delivery_mean

# Добавим столбец "Усредненная сумма хранения на штуку товара"
wb['Усредненная сумма хранения на штуку товара'] = wb_storage_cost_mean

# Добавим столбец "Комиссия поверенного"
wb['Комиссия поверенного'] = wb['Суммарная выручка (WB)'] * 0.015

# Добавим столбец с валовой прибылью
wb['Валовая прибыль'] = wb['Суммарная выручка (WB)'] - (wb['Продажи (WB)'] * wb['Себестоимость']) - (wb['Продажи (WB)'] * wb['Усредненная сумма доставки на штуку товара']) - (wb['Продажи (WB)'] * wb['Усредненная сумма хранения на штуку товара']) - wb['Комиссия поверенного']

In [6]:
# Выгрузим в эксель уникальные артикулы, которые не нашлись в sku
nan_ozon = (
    ozon[ozon['Название линейки'].isna()][['Артикул', 'Название товара или услуги']]
    .rename(columns={"Название товара или услуги": "Название"})
)
nan_wb = wb[wb['Название линейки'].isna()][['Артикул', 'Название']]
nan = nan_ozon.append(nan_wb).drop_duplicates()
nan.to_excel('nan.xlsx')
nan

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_2189/2450536132.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan = nan_ozon.append(nan_wb).drop_duplicates()


,Артикул,Название


In [7]:
# Создадим сгруппированные таблицы по валовой прибыли и бренду
ozon_total = ozon.groupby('Бренд').agg({'Валовая прибыль':'sum'})
wb_total = wb.groupby('Бренд').agg({'Валовая прибыль':'sum'})

In [8]:
# Импортируем листы в эксель
writer = pd.ExcelWriter('Маркетплейсы Валовая прибыль.xlsx', engine='xlsxwriter')
ozon.to_excel(writer, 'Ozon расчеты', index=False)
wb.to_excel(writer, 'WB расчеты', index=False)
ozon_total.to_excel(writer, 'Ozon итого')
wb_total.to_excel(writer, 'WB итого')
writer.save()